# This file compares the teamtat annotation with Extraction performed (Json) of Original Schema

In [1]:
irrelevent_papers = [26,28,29,30,32,33,34,35,38,43,44,45,52,54,55,56,57,58,63,66,68,69,70,78,80,83,84,86,87,88,89,90,91,92,93,94,98,100,101,102,103,104,105,106,108,109,110,111,112,115,116,117,119,121,125,128,129,130,134,136,138,139, 140]
relevent_bad = [1, 18, 20, 25, 27, 41, 51, 61, 71, 76, 135, 141, 145]

relevent_good = [i for i in range(0, 150) if i not in irrelevent_papers and i not in relevent_bad]
print(relevent_good)

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 24, 31, 36, 37, 39, 40, 42, 46, 47, 48, 49, 50, 53, 59, 60, 62, 64, 65, 67, 72, 73, 74, 75, 77, 79, 81, 82, 85, 95, 96, 97, 99, 107, 113, 114, 118, 120, 122, 123, 124, 126, 127, 131, 132, 133, 137, 142, 143, 144, 146, 147, 148, 149]


In [6]:
print(len(irrelevent_papers), len(relevent_bad), len(relevent_good))

63 13 74


In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score
from difflib import SequenceMatcher
import numpy as np
import json
import os
import xml.etree.ElementTree as ET 
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Evaluation schema
data = {
    'perovskite_composition': None,
    'electron_transport_layer': None,
    'hole_transport_layer': None,
    'structure_pin_nip': None,
    'test_1': {
        'stability_type': None,
        'passivating_molecule': None,
        'humidity': None,
        'temperature': None,
        'time': None,
        'control_pce': None,
        'treated_pce': None,
        'control_voc': None,
        'treated_voc': None,
        'efficiency_control': None,
        'efficiency_tret': None,
        'efficiency_cont': None
    }
}


data


{'perovskite_composition': None,
 'electron_transport_layer': None,
 'hole_transport_layer': None,
 'structure_pin_nip': None,
 'test_1': {'stability_type': None,
  'passivating_molecule': None,
  'humidity': None,
  'temperature': None,
  'time': None,
  'control_pce': None,
  'treated_pce': None,
  'control_voc': None,
  'treated_voc': None,
  'efficiency_control': None,
  'efficiency_tret': None,
  'efficiency_cont': None}}

## File Preparation

In [9]:
def str_toJson(string):
    ##The json output from annotation dataframe was not in correct json format
    # We will change the None to null
    if string == None:
        return None
    json_string = string.replace("None", "null")

    try:
        # Try to load the JSON string
        json_object = json.loads(json_string)
        return json_object
    except json.JSONDecodeError as e:
        # Catch JSONDecodeError if the string is not valid JSON
        print(f"Error decoding JSON: {e}")
        print(string)
        print(json_string)
        return None
    except Exception as e:
        # Catch any other exceptions
        print(f"An error occurred: {e}")
        return None

In [10]:
def include_passivating(dictionary):
    ##In extraction json, realized that some extraction has passivating molecule that is NOT included in its stability testing. 
    ## Since passivating molecule (if exist) needs to be in stability testing (nexted dictionary), we will transfer the information and spit out a cleaned dictionary. 
    # print(dictionary)
    if dictionary == None:
        return None
    if "passivating_molecule" in dictionary.keys():
        passivating = dictionary['passivating_molecule']
        del dictionary['passivating_molecule']
        
        for entity in dictionary.keys():
            if entity.startswith('test'):
                # print(i['entity'])
                if type(dictionary[entity]) == dict:
                    if 'passivating_molecule' in dictionary[entity].keys():
                        continue
                    else:
                        # print("Have to include passivating molecule in tests")
                        dictionary[entity]['passivating_molecule'] = passivating
        
    return dictionary

In [11]:
## Convert all numerical data into float for both

#### WORK ON NUMERICAL DATA THAT INCLUDES RANGE "-" 
def convert_numeric(dictionary):
    if dictionary == None:
        return None
    numerical_key = ['time', 'efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    exception_numeric = ['humidity', 'temperature']

    translation_table = str.maketrans('', '', 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!@#$%^&*()')
    for key in dictionary.keys():
        if (key.startswith('test')) & (type(dictionary[key]) == dict):
            for entity in dictionary[key].keys():
                if entity in numerical_key:
                    # print(dictionary[key][entity])
                    if isinstance(dictionary[key][entity], str): 
                        substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                        if len(substitute) != 0:
                            numerical_value = float(substitute)
                            dictionary[key][entity] = numerical_value
                        else:
                            dictionary[key][entity] = None
                elif entity in exception_numeric:
                    if isinstance(dictionary[key][entity], str): 
                        if "-" not in dictionary[key][entity]:
                            # print("regular_case",dictionary[key][entity])
                            substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                            if len(substitute) != 0:
                                numerical_value = float(substitute)
                                dictionary[key][entity] = numerical_value
                            else:
                                dictionary[key][entity] = None
                        # else:
                            
                        #     print(dictionary[key][entity])
        elif ('test_' in key) & (type(dictionary[key]) == dict):
            for entity in dictionary[key].keys():
                if entity in numerical_key:
                    # print(dictionary[key][entity])
                    if isinstance(dictionary[key][entity], str): 
                        substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                        if len(substitute) != 0:
                            numerical_value = float(substitute)
                            dictionary[key][entity] = numerical_value
                        else:
                            dictionary[key][entity] = None
                elif entity in exception_numeric:
                    if isinstance(dictionary[key][entity], str): 
                        if "-" not in dictionary[key][entity]:
                            # print("regular_case",dictionary[key][entity])
                            substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                            if len(substitute) != 0:
                                numerical_value = float(substitute)
                                dictionary[key][entity] = numerical_value
                            else:
                                dictionary[key][entity] = None
                        # else:
                            
                        #     print(dictionary[key][entity])
        elif key in numerical_key:
            if isinstance(dictionary[key], str): 
                substitute = re.sub(r'[^0-9.]', '', dictionary[key][:4])
                if len(substitute) != 0:
                    numerical_value = float(substitute)
                    dictionary[key] = numerical_value
                else:
                    dictionary[key] = None
        elif key in exception_numeric:
            if isinstance(dictionary[key], str): 
                if "-" not in dictionary[key]:
                    # print("regular_case",dictionary[key][entity])
                    substitute = re.sub(r'[^0-9.]', '', dictionary[key][:4])
                    if len(substitute) != 0:
                        numerical_value = float(substitute)
                        dictionary[key] = numerical_value
                    else:
                        dictionary[key] = None




    return dictionary

In [12]:
test_dict = {'test_name': 'ISOS-L', 'temperature': '85', 'humidity': 0, 'time': '1500', 'passivating_molecule': 'phenethylammonium chloride', 'efficiency_tret': '90', 'efficiency_cont': '90', 'control_efficiency': 0, 'treatment_efficiency': 0, 'control_pce': 0, 'treated_pce': 0, 'control_voc': 0, 'treated_voc': 0}

convert_numeric(test_dict)

{'test_name': 'ISOS-L',
 'temperature': 85.0,
 'humidity': 0,
 'time': 1500.0,
 'passivating_molecule': 'phenethylammonium chloride',
 'efficiency_tret': 90.0,
 'efficiency_cont': 90.0,
 'control_efficiency': 0,
 'treatment_efficiency': 0,
 'control_pce': 0,
 'treated_pce': 0,
 'control_voc': 0,
 'treated_voc': 0}

In [13]:
def convert_efficiency(dictionary):
    if dictionary == None:
        return None
    entity_decimal = ['efficiency_cont','efficiency_tret']
    for key in dictionary.keys():
        if (key.startswith('test')) & (type(dictionary[key]) == dict):
            for entity in dictionary[key].keys():
                if (entity in entity_decimal) and (dictionary[key][entity] != None):
                    if dictionary[key][entity] == dictionary[key][entity] > 1:
                        dictionary[key][entity] = dictionary[key][entity] / 100
    return dictionary


    

#### Analyzing these outputs

Annotation notes: 
- THE 4 basic variable that is to compare is PEROVSKITE COMPOSITION, ETL, HTL, STRUCTURE
- Stability entity: efficiency_control is wrong, All value is None, so ignore. 
- Common entity: ['stability_type', 'passivating_molecule', 'humidity', 'temperature', 'time', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc', 'efficiency_tret'] 
    - 'efficiency_cont' are included or not

- the efficiency in extracted data need to be converted to decimals since it is in percentage SOMETIMES
    - 'control_efficiency' and 'treatment_efficiency can be ignored


Extraction notes:
- some extraction has passivating molecule that is NOT included in its stability testing. 

### Loading Teamtat Annotation as dataframe

In [14]:
#Teamtat Annotation
annotation_df = pd.read_csv("data/150_papers_json_update.csv")[["id", "first_num", "output"]]
annotation_df = annotation_df.sort_values(by = ['first_num'])

In [15]:
##Change the format (minor) to be converted to json
annotation_df['output'] = annotation_df['output'].apply(str_toJson)
annotation_df

,id,first_num,output
0,0_54,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
1,1_22,1,"{'perovskite_composition': None, 'electron_tra..."
2,2_75,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
3,3_52,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
4,4_26,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
...,...,...,...
141,145_31,145,{'perovskite_composition': 'Cs0.05(MA0.17FA0.8...
142,146_36,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
143,147_41,147,"{'perovskite_composition': 'FAPbI 3', 'electro..."
144,148_26,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...


In [16]:
len(annotation_df['first_num'].unique())

146

In [17]:
annotation_df['output'] = annotation_df['output'].apply(convert_numeric)

In [18]:
# # Exporting annotation
# annotation_df.to_csv('data/annotation.csv', index=False)

In [19]:
## Get the annotation_df with only relevent good papers. 
annotation_df = annotation_df[annotation_df['first_num'].isin(relevent_good)]
annotation_df

,id,first_num,output
0,0_54,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
2,2_75,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
3,3_52,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
4,4_26,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
5,5_25,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele..."
...,...,...,...
140,#:144_62,144,"{'perovskite_composition': None, 'electron_tra..."
142,146_36,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
143,147_41,147,"{'perovskite_composition': 'FAPbI 3', 'electro..."
144,148_26,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...


In [20]:
annotation_df.iloc[0]['output']

{'perovskite_composition': 'Cs0.05FA0.85MA0.1PbI3',
 'electron_transport_layer': 'C60',
 'hole_transport_layer': '2PACz',
 'structure_pin_nip': 'PIN',
 'test_1': {'stability_type': 'ISOSL3',
  'passivating_molecule': '4-chlorobenzenesulfonate',
  'humidity': 50.0,
  'temperature': 65.0,
  'time': 1200.0,
  'control_pce': 24.0,
  'treated_pce': 26.9,
  'control_voc': None,
  'treated_voc': 1.18,
  'efficiency_cont': None,
  'efficiency_tret': 95.0},
 'test_1_2': {'stability_type': 'ISOSL3',
  'passivating_molecule': '4-chlorobenzenesulfonate',
  'humidity': 50.0,
  'temperature': 85.0,
  'time': 540.0,
  'control_pce': 24.0,
  'treated_pce': 26.9,
  'control_voc': None,
  'treated_voc': None,
  'efficiency_cont': None,
  'efficiency_tret': 87.0},
 'test_2': {'stability_type': 'ISOSD2',
  'passivating_molecule': '4-chlorobenzenesulfonate',
  'humidity': None,
  'temperature': 85.0,
  'time': 1500.0,
  'control_pce': 24.0,
  'treated_pce': 26.9,
  'control_voc': None,
  'treated_voc': Non

### Loading in JSON extraction

In [21]:
def convert_efficiency_key(dictionary):
    if dictionary == None:
        return None
    for key, item in dictionary.items():
        if 'test' in key:
            if item == None:
                continue
            if ("efficiency_cont" in dictionary[key].keys()) | ("efficiency_tret" in dictionary[key].keys()):
                continue
            else:
                if 'retained_proportion_cont' in dictionary[key]:
                    dictionary[key]['efficiency_cont'] = dictionary[key].pop('retained_proportion_cont')
                if 'retained_proportion_tret' in dictionary[key]:
                    dictionary[key]['efficiency_tret'] = dictionary[key].pop('retained_proportion_tret')
                if 'retained_percentage_tret' in dictionary[key]:
                    dictionary[key]['efficiency_tret'] = dictionary[key].pop('retained_percentage_tret')
                if 'retained_percentage_cont' in dictionary[key]:
                    dictionary[key]['efficiency_cont'] = dictionary[key].pop('retained_percentage_cont')
                if 'control_efficiency' in dictionary[key]:
                    dictionary[key]['efficiency_cont'] = dictionary[key].pop('control_efficiency')
                if 'treatment_efficiency' in dictionary[key]:
                    dictionary[key]['efficiency_tret'] = dictionary[key].pop('treatment_efficiency')
            
    return dictionary
            

In [22]:
# structure_pin_nip needs to change

def convert_structure_key(dictionary):
    if dictionary == None:
        return None
    found = 0
    founded = 0
    foundd = 0
    for key, item in dictionary.items():
        if key == "pin_nip_structure":
            found = 1
        if key == "structure_type":
            founded = 1
        if key == "structure":
            foundd = 1
    if found == 1:
        dictionary['structure_pin_nip'] = dictionary['pin_nip_structure']
        dictionary.pop('pin_nip_structure')
    if founded == 1:
        dictionary['structure_pin_nip'] = dictionary['structure_type']
        dictionary.pop('structure_type')
    if foundd == 1:
        dictionary['structure_pin_nip'] = dictionary['structure']
        dictionary.pop('structure')


    return dictionary

In [23]:
## extraction performed by basemodel
# Read JSON from a file
with open("data/output1.json", 'r') as f:
    extraction = json.load(f)

extraction_base = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
extraction_base['paper_num'] = pd.to_numeric(extraction_base['paper_num'])
extraction_base = extraction_base.sort_values('paper_num')
extraction_base['output'] = extraction_base['output'].apply(include_passivating)
extraction_base['output'] = extraction_base['output'].apply(convert_numeric)
extraction_base['output'] = extraction_base['output'].apply(convert_efficiency)
extraction_base




# ## extraction performed by basemodel
# # Read JSON from a file
# with open("data/output1.json", 'r') as f:
#     extraction = json.load(f)

# extraction_base = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
# extraction_base['paper_num'] = pd.to_numeric(extraction_base['paper_num'])
# extraction_base = extraction_base.sort_values('paper_num')
# extraction_base['output'] = extraction_base['output'].apply(include_passivating)
# extraction_base['output'] = extraction_base['output'].apply(convert_efficiency)
# extraction_base['output'] = extraction_base['output'].apply(convert_structure_key)
# extraction_base['output'] = extraction_base['output'].apply(convert_numeric)

# extraction_base

,paper_num,output
77,0,"{'perovskite_composition': 'FAPbI3', 'electron..."
124,1,"{'perovskite_composition': 'FA1-x MAx PbI3', '..."
7,2,"{'perovskite_composition': '(BA)2PbI4', 'elect..."
34,3,{'perovskite_composition': 'Cs0.05 (MA0.10FA0....
30,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
...,...,...
40,145,{'perovskite_composition': 'Cs0.05(MA0.17FA0.8...
87,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
120,147,"{'perovskite_composition': 'FAPbI3', 'electron..."
83,148,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...


In [24]:
## extraction performed by finetuned deepseek 4 bit
# Read JSON from a file
with open("data/deepseek_finetuned_4bit.json", 'r') as f:
    extraction = json.load(f)

extraction_train = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
extraction_train['paper_num'] = pd.to_numeric(extraction_train['paper_num'])
extraction_train = extraction_train.sort_values('paper_num')
extraction_train['output'] = extraction_train['output'].apply(include_passivating)
extraction_train['output'] = extraction_train['output'].apply(convert_efficiency)
extraction_train['output'] = extraction_train['output'].apply(convert_efficiency_key)
extraction_train['output'] = extraction_train['output'].apply(convert_structure_key)
extraction_train['output'] = extraction_train['output'].apply(convert_numeric)
extraction_train

,paper_num,output
111,0,"{'perovskite_composition': 'FAPbI3', 'electron..."
67,1,{'perovskite_composition': 'FA1-x MAx PbI3 (FA...
46,2,{'perovskite_composition': 'FA0.85Cs0.15PbI2.8...
30,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
27,4,"{'perovskite_composition': None, 'electron_tra..."
...,...,...
52,145,{'perovskite_composition': 'Cs0.05(MA0.17FA0.8...
73,146,{'perovskite_composition': 'Cs0.05(FA0.05MA0.9...
102,147,"{'perovskite_composition': 'FAPbI3', 'electron..."
80,148,{'perovskite_composition': 'Cs0.05 FA0.85 MA0....


In [25]:
def escape_internal_quotes(json_string):
    # This regex finds values within quotes and fixes internal unescaped quotes
    return re.sub(r'":\s*"([^"]*?)"', lambda m: '": "' + m.group(1).replace('"', '\\"') + '"', json_string)

In [26]:
def force_fix(json_string):
    '''
    {"perovskite_composition": "Cs0.05(FA0.98MA0.02)0.95Pb(I0.98Br0.02)3", "electron_transport_layer": "6,6"-phenyl-C61-butyric acid methyl ester", "pin_nip_structure": "NIP", "hole_transport_layer": "2-(3,6-dimethoxy-9H-carbazol-9-yl)ethyl phosphonic acid", "test_1": {"test_name": "ISOS-L", "temperature": "25", "time": "1000", "humidity": "50-60", "passivating_molecule": "β-poly(1,1-difluoroethylene)", "control_pce": "22.3", "treated_pce": "24.6", "control_voc": "1.13", "treated_voc": "1.18"}}
    '''
    ## convert {" pattern to ZS
    json_conv = re.sub(r'{\"',"ZS", json_string)
    ## convert ": to ZT
    json_conv = re.sub(r'\":', "ZT", json_conv)
    ## Convert _" to ZP
    json_conv = re.sub(r' \"', "ZP", json_conv)
    ## Convert ", to ZV
    json_conv = re.sub(r'\",Z', "ZVZ", json_conv)
    ## Convert "} to ZQ
    json_conv = re.sub(r'\"}', "ZQ", json_conv)

    ##Perform real conversion interested
    json_conv = json_conv.replace('"', "'")

    #Revert Everything 

    ## convert ZS to {" 
    json_conv = re.sub("ZS", r'{"', json_conv)
    ## convert ": to ZT
    json_conv = re.sub("ZT", r'":', json_conv)
    ## Convert _" to ZP
    json_conv = re.sub("ZP", r' "', json_conv)
    ## Convert ", to ZV
    json_conv = re.sub("ZV", r'",', json_conv)
    ## Convert "} to ZQ
    json_conv = re.sub("ZQ", r'"}', json_conv)

    return json_conv



In [27]:
def str_toJson_8bit(string):
    ##The json output from annotation dataframe was not in correct json format
    # We will change the None to null
    if string == None:
        return None
    
    json_string = string.replace("None", "null")
    json_string = json_string.replace(r'\"', '"')
    # json_string = json_string.replace("'", '"')
    json_string = escape_internal_quotes(json_string)
    json_string = force_fix(json_string)

    json_string = json_string.replace("',\n", '",')
    json_string = json_string.replace("\n", '')
    json_string = json_string.replace("'  }", '"}')
    


    try:
        # Try to load the JSON string
        json_object = json.loads(json_string)
        return json_object
    except json.JSONDecodeError as e:
        # Catch JSONDecodeError if the string is not valid JSON
        print(f"Error decoding JSON: {e}")
        print(json_string)
        return None
    except Exception as e:
        # Catch any other exceptions
        print(f"An error occurred: {e}")
        return None

In [28]:
## extraction performed by finetuned deepseek 8 bit
# Read JSON from a file
with open("data/deepseek_8bit_finetuned.json", 'r') as f:
    extraction = json.load(f)

extraction_train_8 = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
extraction_train_8['paper_num'] = pd.to_numeric(extraction_train_8['paper_num'])
extraction_train_8 = extraction_train_8.sort_values('paper_num')
extraction_train_8['output'] = extraction_train_8['output'].apply(str_toJson_8bit)

extraction_train_8['output'] = extraction_train_8['output'].apply(include_passivating)
# extraction_train_8['output'] = extraction_train_8['output'].apply(convert_efficiency)
extraction_train_8['output'] = extraction_train_8['output'].apply(convert_efficiency_key)
extraction_train_8['output'] = extraction_train_8['output'].apply(convert_structure_key)
extraction_train_8['output'] = extraction_train_8['output'].apply(convert_numeric)

extraction_train_8

Error decoding JSON: Expecting value: line 1 column 29 (char 28)
{"perovskite_composition": ['(BA)2(MA)3Pb4I13", "CH3NH3PbI3'], "electron_transport_layer": null, "hole_transport_layer": null, "pin_nip_structure": "PIN", "test_1": {"passivating_molecule": null, "humidity": null, "temperature": null, "time": null, "control_pce": null, "treated_pce": null, "control_voc": null, "treated_voc": null, "retained_percentage_cont": null, "retained_percentage_tret": null}}


,paper_num,output
130,0,None
79,1,"{'perovskite_composition': 'FA1-xMAxPbI3', 'el..."
54,2,"{'perovskite_composition': None, 'electron_tra..."
36,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
32,4,None
...,...,...
64,145,{'perovskite_composition': 'Cs0.05(MA0.17FA0.8...
86,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
121,147,"{'perovskite_composition': None, 'electron_tra..."
95,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...


In [29]:
for dictionary in extraction_train_8['output']:
    print(dictionary)

None
{'perovskite_composition': 'FA1-xMAxPbI3', 'electron_transport_layer': 'SnO2', 'hole_transport_layer': 'spiro-OMeTAD', 'test_1': {'passivating_molecule': 'phenethylammonium iodide', 'test_name': 'ISOS-L', 'temperature': 85.0, 'time': 500.0, 'humidity': None, 'control_pce': 20.9, 'treated_pce': 23.1, 'control_voc': 1.12, 'treated_voc': 1.16}, 'test_2': {'passivating_molecule': 'phenethylammonium iodide', 'test_name': 'ISOS-L', 'temperature': 100.0, 'time': 40.0, 'humidity': None, 'control_pce': 21.2, 'treated_pce': 22.7, 'control_voc': 1.11, 'treated_voc': 1.16}, 'structure_pin_nip': 'PIN'}
{'perovskite_composition': None, 'electron_transport_layer': None, 'hole_transport_layer': None, 'test_1': {'passivating_molecule': 'Formamidinium lead iodide', 'test_name': 'ISOS-L', 'humidity': 85.0, 'temperature': 25.0, 'time': 1083.0, 'control_pce': 24.0, 'treated_pce': 24.6, 'control_voc': None, 'treated_voc': None, 'efficiency_cont': 2.7, 'efficiency_tret': 95.0}, 'structure_pin_nip': None

In [30]:
def str_toJson_llama(strings):
    ##The json output from annotation dataframe was not in correct json format
    # We will change the None to null
    if strings == None:
        return None
    json_string = strings.replace("None", "null")
    json_string = json_string.replace(r'\"', '"')
    json_string = escape_internal_quotes(json_string)
    json_string = json_string.replace("'", '"')
    json_string = force_fix(json_string)
    
    json_string = json_string.replace("True", "true")
    try:
        # Try to load the JSON string
        json_object = json.loads(json_string)
        return json_object
    except json.JSONDecodeError as e:
        # Catch JSONDecodeError if the string is not valid JSON
        print(f"Error decoding JSON: {e}")
        print(json_string)
        return None
    except Exception as e:
        # Catch any other exceptions
        print(f"An error occurred: {e}")
        return None

In [31]:
## extraction performed by Llama
# Read JSON from a file
with open("data/llama_3b_8bit_fully_nested.json", 'r') as f:
    extraction = json.load(f)

llama = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
llama['paper_num'] = pd.to_numeric(llama['paper_num'])
llama = llama.sort_values('paper_num')
llama['output'] = llama['output'].apply(str_toJson_llama)
llama['output'] = llama['output'].apply(include_passivating)
# llama['output'] = llama['output'].apply(convert_efficiency)
llama['output'] = llama['output'].apply(convert_efficiency_key)
llama['output'] = llama['output'].apply(convert_structure_key)
llama['output'] = llama['output'].apply(convert_numeric)

llama

,paper_num,output
130,0,"{'perovskite_composition': 'FAPbI3', 'electron..."
79,1,"{'perovskite_composition': 'FA1-xMAxPbI3', 'el..."
54,2,{'perovskite_composition': 'Cs0.05(MA0.1FA0.9)...
36,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
32,4,"{'perovskite_composition': 'MAPbI3', 'electron..."
...,...,...
64,145,{'perovskite_composition': 'Cs0.05(MA0.17FA0.8...
86,146,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
121,147,"{'perovskite_composition': 'FAPbI3', 'electron..."
95,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...


## Merging dataframe

In [32]:
evaluate_df_base = annotation_df.merge(extraction_base, left_on='first_num', right_on='paper_num', how = 'left')[["first_num", "output_x",'output_y']]
evaluate_df_base.columns = ['paper_num', 'annotation', 'extracted']
evaluate_df_base["extracted"] = evaluate_df_base["extracted"].apply(lambda x: None if pd.isna(x) else x)

evaluate_df_base_absent = evaluate_df_base[evaluate_df_base['extracted'].isnull()]
##This code below is for dropping row with None In extracted
evaluate_df_base = evaluate_df_base[evaluate_df_base['extracted'].notnull()]

evaluate_df_base

,paper_num,annotation,extracted
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron..."
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': '(BA)2PbI4', 'elect..."
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05 (MA0.10FA0....
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
6,7,{'perovskite_composition': 'Cs0.05(MA)0.16(FA)...,"{'perovskite_composition': 'CsPbI3', 'electron..."
...,...,...,...
69,144,"{'perovskite_composition': None, 'electron_tra...",{'perovskite_composition': 'Cs0.1FA0.6MA0.3Sn0...
70,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
71,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron..."
72,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...


In [33]:
evaluate_df_train = annotation_df.merge(extraction_train, left_on='first_num', right_on='paper_num', how = 'left')[["first_num", "output_x",'output_y']]
evaluate_df_train.columns = ['paper_num', 'annotation', 'extracted']
evaluate_df_train["extracted"] = evaluate_df_train["extracted"].apply(lambda x: None if pd.isna(x) else x)

evaluate_df_train_absent = evaluate_df_train[evaluate_df_train['extracted'].isnull()]
##This code below is for dropping row with None In extracted
evaluate_df_train = evaluate_df_train[evaluate_df_train['extracted'].notnull()]

evaluate_df_train

,paper_num,annotation,extracted
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron..."
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,{'perovskite_composition': 'FA0.85Cs0.15PbI2.8...
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,"{'perovskite_composition': None, 'electron_tra..."
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA(MA)PbI3', 'elec..."
...,...,...,...
69,144,"{'perovskite_composition': None, 'electron_tra...","{'perovskite_composition': None, 'electron_tra..."
70,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,{'perovskite_composition': 'Cs0.05(FA0.05MA0.9...
71,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron..."
72,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs0.05 FA0.85 MA0....


In [34]:
evaluate_df_train8 = annotation_df.merge(extraction_train_8, left_on='first_num', right_on='paper_num')[["paper_num", "output_x",'output_y']]
evaluate_df_train8.columns = ['paper_num', 'annotation', 'extracted']

evaluate_df_train8_absent = evaluate_df_train8[evaluate_df_train8['extracted'].isnull()]
##This code below is for dropping row with None In extracted
evaluate_df_train8 = evaluate_df_train8[evaluate_df_train8['extracted'].notnull()]

evaluate_df_train8

,paper_num,annotation,extracted
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': None, 'electron_tra..."
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA(MA)PbI3', 'elec..."
5,6,"{'perovskite_composition': 'MAPbI 3', 'electro...",{'perovskite_composition': 'PEA2(CH3NH3)n-1Pbn...
6,7,{'perovskite_composition': 'Cs0.05(MA)0.16(FA)...,"{'perovskite_composition': None, 'electron_tra..."
...,...,...,...
69,144,"{'perovskite_composition': None, 'electron_tra...",{'perovskite_composition': 'FA0.98MA0.02Pb(I0....
70,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
71,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': None, 'electron_tra..."
72,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...


In [35]:
evaluate_df_llama = annotation_df.merge(llama, left_on='first_num', right_on='paper_num')[["paper_num", "output_x",'output_y']]
evaluate_df_llama.columns = ['paper_num', 'annotation', 'extracted']

evaluate_df_llama_absent = evaluate_df_llama[evaluate_df_llama['extracted'].isnull()]
##This code below is for dropping row with None In extracted
evaluate_df_llama = evaluate_df_llama[evaluate_df_llama['extracted'].notnull()]

evaluate_df_llama

,paper_num,annotation,extracted
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,"{'perovskite_composition': 'FAPbI3', 'electron..."
1,2,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,{'perovskite_composition': 'Cs0.05(MA0.1FA0.9)...
2,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,"{'perovskite_composition': 'MAPbI3', 'electron..."
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...","{'perovskite_composition': 'FA0.98MA0.02PbI3',..."
...,...,...,...
69,144,"{'perovskite_composition': None, 'electron_tra...",{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
70,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...
71,147,"{'perovskite_composition': 'FAPbI 3', 'electro...","{'perovskite_composition': 'FAPbI3', 'electron..."
72,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...


In [36]:
# evaluate_df.to_csv('merged.csv', index=False)

In [37]:
# for row in evaluate_df_train.itertuples():
#     label_value = row.annotation
#     extracted_value = row.extracted

#     print(label_value)
#     print(extracted_value)

#### Side: Inspecting missingness

There are 74 relevent good paper in annotation. Some were not extracted for certain papers. Inspecting...

In [38]:
evaluate_df_base_absent

,paper_num,annotation,extracted
4,5,"{'perovskite_composition': 'FA(MA)PbI 3', 'ele...",None
5,6,"{'perovskite_composition': 'MAPbI 3', 'electro...",None
13,14,{'perovskite_composition': '2-phenylethylammon...,None
14,15,{'perovskite_composition': 'Cs0.05(FA5/6MA1/6)...,None
24,37,{'perovskite_composition': '(Cs0.2FA0.8Pb(I0.6...,None
26,40,{'perovskite_composition': 'Cs0.03(FA0.90MA0.1...,None
31,49,"{'perovskite_composition': None, 'electron_tra...",None
40,72,{'perovskite_composition': 'Cs0.05FA0.8MA0.15P...,None
46,81,{'perovskite_composition': 'Cs0.05FA0.81MA0.14...,None
48,85,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,None


In [39]:
evaluate_df_train_absent

,paper_num,annotation,extracted
6,7,{'perovskite_composition': 'Cs0.05(MA)0.16(FA)...,None
9,10,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,None
10,11,"{'perovskite_composition': 'FAPbI3', 'electron...",None
28,46,{'perovskite_composition': 'CsPbI0.05[(FAPbI3)...,None
36,62,"{'perovskite_composition': 'MAPbI 3', 'electro...",None
37,64,"{'perovskite_composition': 'MAPbI3', 'electron...",None
38,65,"{'perovskite_composition': 'MAPbI3', 'electron...",None
43,75,{'perovskite_composition': '[(FAPbI 3 ) 0.8(MA...,None
44,77,"{'perovskite_composition': 'FA0.9Cs0.1PbI3', '...",None
56,118,{'perovskite_composition': 'Cs0.03(FA0.97MA0.0...,None


In [40]:
evaluate_df_train8_absent

,paper_num,annotation,extracted
0,0,{'perovskite_composition': 'Cs0.05FA0.85MA0.1P...,None
3,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...,None
21,24,"{'perovskite_composition': 'CsPbI 3', 'electro...",None
29,47,{'perovskite_composition': 'Cs0.05FA0.81MA0.14...,None


In [41]:
evaluate_df_llama_absent

,paper_num,annotation,extracted
6,7,{'perovskite_composition': 'Cs0.05(MA)0.16(FA)...,None
44,77,"{'perovskite_composition': 'FA0.9Cs0.1PbI3', '...",None


## Evaluation

- We need precision and recall for EACH variable
- For each variable, calculate the F1 score - There is F1 score for each variable
- Take a weighted average ***For now, just take the average.

In [42]:
def tests_comparison(stability_annotated, label_dict, stability_extracted, extract_dict):
    # print(stability_annotated, label_dict, stability_extracted, extract_dict)
    stability_entity_annotated = ['stability_type', 'passivating_molecule', 'temperature', 'time', 'humidity', 'efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    stability_entity_extracted = ['test_name', 'passivating_molecule', 'temperature', 'time', 'humidity', 'control_efficiency', 'treatment_efficiency', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    
    # print(f"stability_annotated{stability_annotated}")
    # print(f"label_dict{label_dict}")
    # print(f"stability_extracted{stability_extracted}")
    # print(f"extract_dict{extract_dict}")



    compared_metric = []
    numeric_data_annotated = []
    numeric_data_extracted = []
    for entity_i in range(len(stability_entity_annotated)):
        if entity_i <= 1:
            if stability_entity_extracted[entity_i] not in extract_dict.keys():
                extract_dict[stability_entity_extracted[entity_i]] = None

            if (label_dict[stability_entity_annotated[entity_i]] == None) | (extract_dict[stability_entity_extracted[entity_i]] == None):
                compared_metric.append(None)
            else:
                ##Text entity, perform Sequence Matcher 
                compared = SequenceMatcher(None, label_dict[stability_entity_annotated[entity_i]], extract_dict[stability_entity_extracted[entity_i]]).ratio()
                # print(compared)
                if entity_i == 0:
                    if compared > 0.9:
                        compared_metric.append(1)
                    else:
                        compared_metric.append(0)
                else:
                    compared_metric.append(compared)
        else:
            if stability_entity_extracted[entity_i] not in extract_dict.keys():
                extract_dict[stability_entity_extracted[entity_i]] = 0
            elif extract_dict[stability_entity_extracted[entity_i]] == None:
                extract_dict[stability_entity_extracted[entity_i]] = 0

            if stability_entity_annotated[entity_i] not in label_dict.keys():
                label_dict[stability_entity_annotated[entity_i]] = 0
            elif label_dict[stability_entity_annotated[entity_i]] == None:
                label_dict[stability_entity_annotated[entity_i]] = 0

                
            numeric_data_annotated.append(label_dict[stability_entity_annotated[entity_i]])
            numeric_data_extracted.append(extract_dict[stability_entity_extracted[entity_i]])

    if isinstance(numeric_data_extracted[0], list):
        ##There was one column with two temperature recorded as a list (probably thermal cycling)
        numeric_data_extracted[0] = numeric_data_extracted[0][1]

    # print(numeric_data_annotated, numeric_data_extracted)

    numeric_annotated_clean = []
    numeric_extracted_clean = []
    ##Clean the numeric data to skip any strings
    for i in range(len(numeric_data_annotated)):
        if (type(numeric_data_annotated[i]) == str) | (type(numeric_data_extracted[i]) == str):
            continue
        else:
            numeric_annotated_clean.append(numeric_data_annotated[i])
            numeric_extracted_clean.append(numeric_data_extracted[i])

    cos_sim = cosine_similarity([numeric_annotated_clean], [numeric_extracted_clean])
    compared_metric.append(cos_sim[0][0])
    
    return compared_metric   


In [43]:
def entity_comparison(entity, label, extracted_dict, text_similarity_threshold = 0.75, numerical_tolerance = 0.027):
    '''
    The tolarance of 2.7% was what was reasonable looking at the absolute difference
    treated_voc 1.18, 1.149, absolute difference 0.026271186440677895

    The text similarity were set to 75% due to the structure example
    FP, NIP, n-i-p, 0.75
    This should be positive
    
    '''
    text_entity = ['stability_type', 'passivating_molecule']
    numerical_entity = ['time', 'efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    numerical_exception = ['temperature', 'humidity']

    if entity in text_entity:
        key_to_check = "test_name" if entity == "stability_type" else entity

        # If the key is missing in the extracted annotation, return False Negative
        if (label[entity]!=None) & (extracted_dict[key_to_check]==None):
            # print(f"FN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "FN"
        elif (label[entity]==None) & (extracted_dict[key_to_check]!=None):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "TN"

        label_data = label.get(entity, "")
        extract_data = extracted_dict.get(key_to_check, "")

        # Convert lists to strings if necessary
        if isinstance(label_data, list):
            label_data = " ".join(map(str, label_data))  # Convert list to string
        if isinstance(extract_data, list):
            extract_data = " ".join(map(str, extract_data))  # Convert list to string

        # Ensure values are strings
        if not isinstance(label_data, str) or not isinstance(extract_data, str):
            # print(f"FP, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "FP"  # If data is still not a string, return False Positive

        # Compute similarity score
        similarity = SequenceMatcher(None, label_data.lower(), extract_data.lower()).ratio()

        if similarity > text_similarity_threshold:
            # print(f"TP,{entity} {label_data}, {extract_data}")
            return 'TP'
        else:
            # print(f"FP,{entity} {label_data}, {extract_data}, {similarity}")
            return "FP"
    elif entity in numerical_entity:
        # key_to_check = "control_efficiency" if entity == "efficiency_cont" else ("treatment_efficiency" if entity == "efficiency_tret" else entity)

        # print(f"annotated{label[entity]}")
        # print(f"extracted{extracted_dict[entity]}")
        print(entity)
        print(extracted_dict)
        if entity not in extracted_dict.keys():
            extracted_dict[entity] = 0
        if extracted_dict[entity] == None:
            extracted_dict[entity] = 0

        # If the key is missing in the extracted annotation, return False Negative
        if (label[entity]!=0) & ((extracted_dict[entity]==0) | (entity not in extracted_dict.keys())):
            # print(f"FN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "FN"
        elif (label[entity]==0) & (extracted_dict[entity]!=0):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "TN"
        elif (label[entity]==0) & (extracted_dict[entity]==0):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "TN"


        if isinstance(extracted_dict[entity], list):
            ##There was one column with two temperature recorded as a list (probably thermal cycling)
            extracted_dict[entity] = extracted_dict[entity][1]

        # print(entity)
        # print(type(extracted_dict[entity]))
        # print(label[entity])
        # Apply numerical tolerance check
        if (abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )<= numerical_tolerance:

            # print(f"Numerical differences matched: {entity} {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
            return "TP"  # True Positive: Correct numerical extraction
        else:

            # print(f"Numerical differences no match: {entity}, {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
            return "FP"  # False Positive: Incorrect numerical extraction    
    else: 
        if isinstance(label[entity], (float, int)):
            if extracted_dict[entity] == None:
                extracted_dict[entity] = 0

            # If the key is missing in the extracted annotation, return False Negative
            if (label[entity]!=0) & ((extracted_dict[entity]==0) | (entity not in extracted_dict.keys())):
                # print(f"FN, {label_annotation[id]}, {extraction_annotation[entity]}")
                return "FN"
            elif (label[entity]==0) & (extracted_dict[entity]!=0):
                # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
                return "TN"
            elif (label[entity]==0) & (extracted_dict[entity]==0):
                # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
                return "TN"


            if isinstance(extracted_dict[entity], list):
                ##There was one column with two temperature recorded as a list (probably thermal cycling)
                extracted_dict[entity] = extracted_dict[entity][1]
            
            print(entity)
            print(type(extracted_dict[entity]))
            print(extracted_dict)
            print(label)
            if type(extracted_dict[entity]) == str:
                ## This is a case where the extraction was string in range of number and annotation were a number. 
                extracted_first_num = float(extracted_dict[entity][:2])
                extracted_second_num = float(extracted_dict[entity][3:])
                if extracted_first_num < label[entity] < extracted_second_num:
                    return "TP"
                else:
                    return "FP"

            # Apply numerical tolerance check
            if (abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )<= numerical_tolerance:

                # print(f"Numerical differences matched: {entity} {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
                return "TP"  # True Positive: Correct numerical extraction
            else:

                # print(f"Numerical differences no match: {entity}, {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
                return "FP"  # False Positive: Incorrect numerical extraction    
        else:
            # print(label[entity], type(label[entity]))
            if extracted_dict[entity] == None:
                extracted_dict[entity] = 0
            
            if ((extracted_dict[entity]==0) | (entity not in extracted_dict.keys())):
                # print(f"FN, {label_annotation[id]}, {extraction_annotation[entity]}")
                return "FN"

            if isinstance(extracted_dict[entity], list):
                ##There was one column with two temperature recorded as a list (probably thermal cycling)
                extracted_dict[entity] = extracted_dict[entity][1]
            
            if isinstance(extracted_dict[entity], str):
                ##Label is str, extraction is str, so perform text similarity
                similarity = SequenceMatcher(None, label[entity].lower(), extracted_dict[entity].lower()).ratio()
                if similarity > text_similarity_threshold:
                    # print(f"TP, {label_data}, {extract_data}, {similarity}")
                    return 'TP'
                else:
                    # print(f"FP, {label_data}, {extract_data}, {similarity}")
                    return "FP"
            else:
                if "+" in label[entity]:
                    ### This is where the data is value+-ME
                    # print(label[entity].split("+-"))
                    value = float(label[entity].split("+-")[0])
                    margin_error = float(label[entity].split("+-")[1])
                    range = (value-margin_error, value-margin_error)
                    if (range[0]<= extracted_dict[entity]) & (extracted_dict[entity]<=range[1]):
                        # print(f"TP, {label_data}, {extract_data}, {similarity}")
                        return 'TP'
                    else:
                        # print(f"FP, {label_data}, {extract_data}, {similarity}")
                        return "FP"
                else:
                    ### 30-50 
                    lower = float(label[entity].split("-")[0])
                    upper = float(label[entity].split("-")[1])
                    if (lower<= extracted_dict[entity]) & (extracted_dict[entity]<=upper):
                        # print(f"TP, {label_data}, {extract_data}, {similarity}")
                        return 'TP'
                    else:
                        # print(f"FP, {label_data}, {extract_data}, {similarity}")
                        return "FP"


In [44]:
def safe_division(numerator, denominator):
    """Returns division result, or 0 if the denominator is zero."""
    return numerator / denominator if denominator != 0 else 0

In [45]:
def text_comparison(id, label_annotation, extraction_annotation, text_similarity_threshold=0.75):
    """Compares text values using string similarity matching.
    - THE 4 basic variable that is to compare is PEROVSKITE COMPOSITION, ETL, HTL, STRUCTURE
    """

    # Handle special case for structure_pin_nip
    # key_to_check = "pin_nip_structure" if id == "structure_pin_nip" else id
    print(id)
    print(extraction_annotation)
    if extraction_annotation == None:
        return "FN"
    print(id)
    print(extraction_annotation)
    # If the key is missing in the extracted annotation, return False Negative
    if (id not in extraction_annotation.keys()):
        return "FN"

    if (extraction_annotation[id]==None):
        # print(f"FN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
        return "FN"
    elif (label_annotation[id]==None) & (extraction_annotation[id]!=None):
        # print(f"TN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
        return "TN"

    label_data = label_annotation.get(id, "")
    if id == 'electron_transport_layer' and label_data == "buckminsterfullerene":
        label_data = 'C60'
    extract_data = extraction_annotation.get(id, "")

    # Convert lists to strings if necessary
    if isinstance(label_data, list):
        label_data = " ".join(map(str, label_data))  # Convert list to string
    if isinstance(extract_data, list):
        extract_data = " ".join(map(str, extract_data))  # Convert list to string

    # Ensure values are strings
    if not isinstance(label_data, str) or not isinstance(extract_data, str):
        # print(f"FP, {label_annotation[id]}, {extraction_annotation[id]}")
        return "FP"  # If data is still not a string, return False Positive

    # Compute similarity score
    similarity = SequenceMatcher(None, label_data.lower(), extract_data.lower()).ratio()

    if similarity > text_similarity_threshold:
        # print(f"TP, {label_data}, {extract_data}, {similarity}")
        return 'TP'
    else:
        # print(f"FP, {label_data}, {extract_data}, {similarity}")
        return "FP"


In [46]:
def compare_json(df):
    """
    Compare labeled and extracted JSON data for correctness.

    TP: Correct value extracted by LLM.
    FN: LLM didn't extract this variable.
    FP: LLM extracted a value, but it was incorrect.
    TN: LLM halucinated and returned value that was not extracted
    """
    
    text_variables = ['perovskite_composition', 'electron_transport_layer', 'hole_transport_layer', 'structure_pin_nip']

    
    stability_entity_annotated = ['stability_type', 'temperature', 'time', 'humidity', 'passivating_molecule', 'efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    stability_entity_extracted = ['test_name', 'temperature', 'time', 'humidity', 'passivating_molecule','control_efficiency', 'treatment_efficiency', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    
    # Initialize comparison dictionaries
    text_dict = {var: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for var in text_variables}
    stability_dict = {var: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for var in stability_entity_annotated}

    for row in df.itertuples():       
        label_value = row.annotation
        extracted_value = row.extracted

        # print(label_value)
        # print(extracted_value)

        for id, label in label_value.items():
            if ('test' in id) and (isinstance(label_value[id], dict)):
                ##Plan for stability test evaluation
                '''
                For each stability condition in annotation, 
                    Pair them with stability condition in extracted
                        With stability of annotation and extraction, use function tests_comparison that returns how similar 2 stabilities are
                    
                Once all the pair is calculated, find the stability name of extraction that was closest to annotation stability. 

                Using this dictionary, we will increment FN, FP, TN, TP for each element of the entity.
                '''
                matched = 0
                stability_match = {}
                
                if extracted_value != None:
                    for extract_id, extract_label in extracted_value.items():
                        if ('test' in extract_id) and (isinstance(extracted_value[extract_id], dict)):
                            matched += 1
                            match_list = tests_comparison(id, label, extract_id, extract_label)
                            match_list = [0 if item is None else item for item in match_list]
                            # print(extracted_value[extract_id])
                            # print(match_list)
                            stability_match[extract_id] = match_list
        
                if matched == 0:
                    #No stability were extracted, we will add stability_unmatched
                        ##We need to account for if there was NO stability extracted. 
                    for key in stability_dict:
                        if 'FN' in stability_dict[key]:
                            stability_dict[key]['FN'] += 1
                else:
                    stability_match_mean = {stability: np.mean(lis) for stability, lis in stability_match.items()}
                    ##Max key is a test_... KEY in Extraction that BEST matched the testID in annotation. 
                    max_key = max(stability_match_mean, key=stability_match_mean.get)  
                    # print(extracted_value[max_key])
                    ##Now, I need to compare each entity in that found max_key and fill in that FN, dictionary.
                    for entity in label_value[id].keys():
                        if entity == 'efficiency_control':
                            continue
                        if entity == 'perovskite_molecule':
                            continue
                        entity_result = entity_comparison(entity, label, extracted_value[max_key])
                        stability_dict[entity][entity_result] += 1  
            else:  
                result = text_comparison(id, label_value, extracted_value)
                
                text_dict[id][result] += 1


    # Merge all results
    combined_dict = {**text_dict, **stability_dict}
    # print("Performance for each variable in dictionary:", combined_dict)

    # Compute precision, recall, and F1-score
    variable_list, precision_list, recall_list, f1_list = [], [], [], []
    for variable, performance in combined_dict.items():
        TP, FP, FN = performance["TP"], performance["FP"], performance["FN"]
        
        precision = safe_division(TP, TP + FP)
        recall = safe_division(TP, TP + FN)
        f1 = safe_division(2 * precision * recall, precision + recall)

        variable_list.append(variable)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return combined_dict, variable_list, precision_list, recall_list, f1_list

In [47]:
dict_result_base, variables_base, precisions_base, recalls_base, f1s_base = compare_json(evaluate_df_base)

perovskite_composition
{'perovskite_composition': 'FAPbI3', 'electron_transport_layer': 'TiO2', 'structure_pin_nip': 'PIN', 'hole_transport_layer': 'TiO2', 'test_1': {'stability_type': 'ISOS-D', 'temperature': 85, 'time': 1050, 'humidity': None, 'efficiency_cont': 0.257, 'efficiency_tret': 0.24, 'passivating_molecule': 'Cyclohexylmethylammonium iodide (CMAI)', 'control_pce': 25.7, 'control_voc': 1.17, 'treated_pce': 24, 'treated_voc': 1.15}, 'test_2': {'stability_type': 'ISOS-L', 'temperature': None, 'time': None, 'humidity': None, 'efficiency_cont': None, 'efficiency_tret': None, 'passivating_molecule': 'Cyclohexylmethylammonium iodide (CMAI)', 'control_pce': 25.7, 'control_voc': 1.17, 'treated_pce': 24, 'treated_voc': 1.15}, 'test_3': {'stability_type': 'ISOS-LT', 'temperature': 50, 'time': 500, 'humidity': 40.0, 'efficiency_cont': None, 'efficiency_tret': 0.95, 'passivating_molecule': 'Cyclohexylmethylammonium iodide (CMAI)', 'control_pce': 25.7, 'control_voc': 1.17, 'treated_pce': 

In [48]:
dict_result_train, variables_train, precisions_train, recalls_train, f1s_train = compare_json(evaluate_df_train)

perovskite_composition
{'perovskite_composition': 'FAPbI3', 'electron_transport_layer': 'Spiro-OMeTAD', 'hole_transport_layer': 'C60', 'structure_pin_nip': 'PIN', 'test_1': {'stability_type': 'ISOS-L-3', 'humidity': 50, 'temperature': 65, 'time': None, 'passivating_molecule': 'cyclohexylmethylammonium iodide', 'control_pce': 24.74, 'treated_pce': 25.74, 'control_voc': 1.18, 'treated_voc': 1.15, 'efficiency_cont': 0.95, 'efficiency_tret': 0.95}}
perovskite_composition
{'perovskite_composition': 'FAPbI3', 'electron_transport_layer': 'Spiro-OMeTAD', 'hole_transport_layer': 'C60', 'structure_pin_nip': 'PIN', 'test_1': {'stability_type': 'ISOS-L-3', 'humidity': 50, 'temperature': 65, 'time': None, 'passivating_molecule': 'cyclohexylmethylammonium iodide', 'control_pce': 24.74, 'treated_pce': 25.74, 'control_voc': 1.18, 'treated_voc': 1.15, 'efficiency_cont': 0.95, 'efficiency_tret': 0.95}}
electron_transport_layer
{'perovskite_composition': 'FAPbI3', 'electron_transport_layer': 'Spiro-OMeTA

In [49]:
dict_result_train_8, variables_train_8, precisions_train_8, recalls_train_8, f1s_train_8 = compare_json(evaluate_df_train8)

perovskite_composition
{'perovskite_composition': None, 'electron_transport_layer': None, 'hole_transport_layer': None, 'test_1': {'passivating_molecule': 'Formamidinium lead iodide', 'test_name': 'ISOS-L', 'humidity': 85.0, 'temperature': 25.0, 'time': 1083.0, 'control_pce': 24.0, 'treated_pce': 24.6, 'control_voc': None, 'treated_voc': None, 'efficiency_cont': 2.7, 'efficiency_tret': 95.0}, 'structure_pin_nip': None}
perovskite_composition
{'perovskite_composition': None, 'electron_transport_layer': None, 'hole_transport_layer': None, 'test_1': {'passivating_molecule': 'Formamidinium lead iodide', 'test_name': 'ISOS-L', 'humidity': 85.0, 'temperature': 25.0, 'time': 1083.0, 'control_pce': 24.0, 'treated_pce': 24.6, 'control_voc': None, 'treated_voc': None, 'efficiency_cont': 2.7, 'efficiency_tret': 95.0}, 'structure_pin_nip': None}
electron_transport_layer
{'perovskite_composition': None, 'electron_transport_layer': None, 'hole_transport_layer': None, 'test_1': {'passivating_molecule

In [50]:
dict_result_llama, variables_llama, precisions_llama, recalls_llama, f1s_llama = compare_json(evaluate_df_llama)

perovskite_composition
{'perovskite_composition': 'FAPbI3', 'electron_transport_layer': 'C60', 'hole_transport_layer': 'Spiro-OMeTAD', 'test_1': {'stability_type': 'ISOSLT', 'passivating_molecule': 'CMAI', 'humidity': '20-30', 'temperature': 65.0, 'time': 1200.0, 'control_pce': 21.0, 'treated_pce': 24.7, 'control_voc': 1.18, 'treated_voc': 1.18, 'test_name': 'ISOSLT', 'efficiency_tret': 100.0, 'efficiency_cont': 95.0}, 'structure_pin_nip': 'NIP'}
perovskite_composition
{'perovskite_composition': 'FAPbI3', 'electron_transport_layer': 'C60', 'hole_transport_layer': 'Spiro-OMeTAD', 'test_1': {'stability_type': 'ISOSLT', 'passivating_molecule': 'CMAI', 'humidity': '20-30', 'temperature': 65.0, 'time': 1200.0, 'control_pce': 21.0, 'treated_pce': 24.7, 'control_voc': 1.18, 'treated_voc': 1.18, 'test_name': 'ISOSLT', 'efficiency_tret': 100.0, 'efficiency_cont': 95.0}, 'structure_pin_nip': 'NIP'}
electron_transport_layer
{'perovskite_composition': 'FAPbI3', 'electron_transport_layer': 'C60', '

In [51]:
dict_result_base

{'perovskite_composition': {'TP': 43, 'FP': 17, 'FN': 1, 'TN': 1},
 'electron_transport_layer': {'TP': 8, 'FP': 37, 'FN': 7, 'TN': 10},
 'hole_transport_layer': {'TP': 20, 'FP': 22, 'FN': 10, 'TN': 10},
 'structure_pin_nip': {'TP': 26, 'FP': 17, 'FN': 11, 'TN': 8},
 'stability_type': {'TP': 0, 'FP': 20, 'FN': 109, 'TN': 0},
 'temperature': {'TP': 37, 'FP': 19, 'FN': 16, 'TN': 57},
 'time': {'TP': 49, 'FP': 28, 'FN': 28, 'TN': 24},
 'humidity': {'TP': 20, 'FP': 7, 'FN': 25, 'TN': 77},
 'passivating_molecule': {'TP': 25, 'FP': 83, 'FN': 19, 'TN': 2},
 'efficiency_cont': {'TP': 0, 'FP': 37, 'FN': 22, 'TN': 70},
 'efficiency_tret': {'TP': 0, 'FP': 67, 'FN': 42, 'TN': 20},
 'control_pce': {'TP': 33, 'FP': 39, 'FN': 22, 'TN': 35},
 'treated_pce': {'TP': 59, 'FP': 37, 'FN': 20, 'TN': 13},
 'control_voc': {'TP': 37, 'FP': 11, 'FN': 22, 'TN': 59},
 'treated_voc': {'TP': 58, 'FP': 10, 'FN': 19, 'TN': 42}}

In [52]:
dict_result_train

{'perovskite_composition': {'TP': 33, 'FP': 25, 'FN': 5, 'TN': 1},
 'electron_transport_layer': {'TP': 16, 'FP': 32, 'FN': 1, 'TN': 15},
 'hole_transport_layer': {'TP': 26, 'FP': 23, 'FN': 2, 'TN': 13},
 'structure_pin_nip': {'TP': 16, 'FP': 28, 'FN': 6, 'TN': 14},
 'stability_type': {'TP': 0, 'FP': 27, 'FN': 103, 'TN': 0},
 'temperature': {'TP': 31, 'FP': 22, 'FN': 0, 'TN': 77},
 'time': {'TP': 26, 'FP': 34, 'FN': 39, 'TN': 31},
 'humidity': {'TP': 13, 'FP': 8, 'FN': 23, 'TN': 86},
 'passivating_molecule': {'TP': 44, 'FP': 72, 'FN': 8, 'TN': 6},
 'efficiency_cont': {'TP': 5, 'FP': 31, 'FN': 13, 'TN': 81},
 'efficiency_tret': {'TP': 18, 'FP': 84, 'FN': 0, 'TN': 28},
 'control_pce': {'TP': 42, 'FP': 39, 'FN': 3, 'TN': 46},
 'treated_pce': {'TP': 57, 'FP': 57, 'FN': 1, 'TN': 15},
 'control_voc': {'TP': 47, 'FP': 12, 'FN': 5, 'TN': 66},
 'treated_voc': {'TP': 56, 'FP': 23, 'FN': 3, 'TN': 48}}

In [53]:
dict_result_train_8

{'perovskite_composition': {'TP': 38, 'FP': 16, 'FN': 14, 'TN': 2},
 'electron_transport_layer': {'TP': 19, 'FP': 27, 'FN': 10, 'TN': 14},
 'hole_transport_layer': {'TP': 28, 'FP': 20, 'FN': 9, 'TN': 13},
 'structure_pin_nip': {'TP': 26, 'FP': 19, 'FN': 9, 'TN': 16},
 'stability_type': {'TP': 42, 'FP': 30, 'FN': 57, 'TN': 19},
 'temperature': {'TP': 33, 'FP': 19, 'FN': 8, 'TN': 88},
 'time': {'TP': 51, 'FP': 53, 'FN': 8, 'TN': 36},
 'humidity': {'TP': 22, 'FP': 14, 'FN': 12, 'TN': 100},
 'passivating_molecule': {'TP': 44, 'FP': 82, 'FN': 13, 'TN': 9},
 'efficiency_cont': {'TP': 4, 'FP': 11, 'FN': 40, 'TN': 93},
 'efficiency_tret': {'TP': 27, 'FP': 30, 'FN': 58, 'TN': 33},
 'control_pce': {'TP': 46, 'FP': 36, 'FN': 13, 'TN': 53},
 'treated_pce': {'TP': 64, 'FP': 55, 'FN': 9, 'TN': 20},
 'control_voc': {'TP': 51, 'FP': 11, 'FN': 14, 'TN': 72},
 'treated_voc': {'TP': 58, 'FP': 24, 'FN': 13, 'TN': 53}}

In [54]:
dict_result_llama

{'perovskite_composition': {'TP': 35, 'FP': 28, 'FN': 7, 'TN': 2},
 'electron_transport_layer': {'TP': 22, 'FP': 25, 'FN': 10, 'TN': 15},
 'hole_transport_layer': {'TP': 27, 'FP': 23, 'FN': 10, 'TN': 12},
 'structure_pin_nip': {'TP': 31, 'FP': 16, 'FN': 8, 'TN': 17},
 'stability_type': {'TP': 38, 'FP': 35, 'FN': 55, 'TN': 18},
 'temperature': {'TP': 29, 'FP': 25, 'FN': 6, 'TN': 86},
 'time': {'TP': 44, 'FP': 60, 'FN': 5, 'TN': 37},
 'humidity': {'TP': 21, 'FP': 17, 'FN': 8, 'TN': 100},
 'passivating_molecule': {'TP': 51, 'FP': 79, 'FN': 11, 'TN': 5},
 'efficiency_cont': {'TP': 3, 'FP': 19, 'FN': 32, 'TN': 92},
 'efficiency_tret': {'TP': 22, 'FP': 36, 'FN': 54, 'TN': 34},
 'control_pce': {'TP': 50, 'FP': 38, 'FN': 8, 'TN': 50},
 'treated_pce': {'TP': 59, 'FP': 55, 'FN': 15, 'TN': 17},
 'control_voc': {'TP': 50, 'FP': 15, 'FN': 9, 'TN': 72},
 'treated_voc': {'TP': 60, 'FP': 25, 'FN': 9, 'TN': 52}}

## Calculate Macro f1 score

In [55]:
def macro_f1(f1_list, weight = None):
    if weight == None:
        #If no weight given, do unweighted average of f1 score
        return sum(f1_list) / len(f1_list)
    total_f1 = 0
    for i in range(len(f1_list)):
        total_f1 += (f1_list[i] * weight[i])
    return total_f1 / sum(weight)
    


In [56]:
# Define column names
columns = ['Macro F1 score weight distribution', 'Base Deepseek 8bit', 'Fine-Tuned Deepseek 4 bit', 'Fine-Tuned Deepseek 8 bit', 'Llama 3 billion parameter']

# Create a DataFrame with NaN values
df_f1scores = pd.DataFrame(np.nan, index=[0, 1, 2, 3, 4, 5, 6], columns=columns)
df_f1scores

,Macro F1 score weight distribution,Base Deepseek 8bit,Fine-Tuned Deepseek 4 bit,Fine-Tuned Deepseek 8 bit,Llama 3 billion parameter
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN


In [57]:
## Unweighted
macro_train_0 = macro_f1(f1s_train)
macro_train_0


0.5298073043022984

In [58]:
macro_base_0 = macro_f1(f1s_base)
macro_base_0

0.4789646987171249

In [59]:
macro_train8_0 = macro_f1(f1s_train_8)
macro_train8_0

0.5909955214633946

In [60]:
macro_llama_0 = macro_f1(f1s_llama)
macro_llama_0

0.5826619472599517

In [61]:
## unweighted row value
unweighted = ['Macro F1 score with equal weight', macro_base_0, macro_train_0, macro_train8_0, macro_llama_0]
df_f1scores.loc[0] = unweighted


In [62]:
variables_train

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'passivating_molecule',
 'efficiency_cont',
 'efficiency_tret',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc']

In [63]:
weights_1 = [1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1]

In [64]:
macro_train_1 = macro_f1(f1s_train, weight = weights_1)
macro_train_1

0.4782157304139091

In [65]:
macro_base_1 = macro_f1(f1s_base, weight = weights_1)
macro_base_1

0.431204226440232

In [66]:
macro_train8_1 = macro_f1(f1s_train_8, weight = weights_1)
macro_train8_1

0.5636215010332426

In [67]:
macro_llama_1 = macro_f1(f1s_llama, weight = weights_1)
macro_llama_1

0.5469093506973081

In [68]:
## Heavier weight on stability value
first_f1 = ['Heavier weight on stability', macro_base_1, macro_train_1, macro_train8_1, macro_llama_1]
df_f1scores.loc[1] = first_f1

In [69]:
variables_train

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'passivating_molecule',
 'efficiency_cont',
 'efficiency_tret',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc']

In [70]:
weights_2 = [2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1]

In [71]:
macro_train_2 = macro_f1(f1s_train, weight = weights_2)
macro_train_2

0.5405449970412426

In [72]:
macro_base_2 = macro_f1(f1s_base, weight = weights_2)
macro_base_2

0.4906281574161612

In [73]:
macro_train8_2 = macro_f1(f1s_train_8, weight = weights_2)
macro_train8_2

0.5939139233522358

In [74]:
macro_llama_2 = macro_f1(f1s_llama, weight = weights_2)
macro_llama_2

0.5918213894306475

In [75]:
## Heavier weight on perovskite structure
first_f2 = ['Heavier weight on perovskite structure', macro_base_2, macro_train_2, macro_train8_2, macro_llama_2]
df_f1scores.loc[2] = first_f2

In [76]:
variables_train

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'passivating_molecule',
 'efficiency_cont',
 'efficiency_tret',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc']

In [77]:
weights_3 = [1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2]

In [78]:
macro_train_3 = macro_f1(f1s_train, weight = weights_3)
macro_train_3

0.542934531365774

In [79]:
macro_base_3 = macro_f1(f1s_base, weight = weights_3)
macro_base_3

0.48920201224780807

In [80]:
macro_train8_3 = macro_f1(f1s_train_8, weight = weights_3)
macro_train8_3

0.5927204970263588

In [81]:
macro_llama_3 = macro_f1(f1s_llama, weight = weights_3)
macro_llama_3

0.5802303546993196

In [82]:
## Heavier weight on numeric data
first_f3 = ['Heavier weight on numeric data', macro_base_3, macro_train_3, macro_train8_3, macro_llama_3]
df_f1scores.loc[3] = first_f3

In [83]:
variables_train

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'passivating_molecule',
 'efficiency_cont',
 'efficiency_tret',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc']

In [84]:
weights_4 = [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]

In [85]:
macro_train_4 = macro_f1(f1s_train, weight = weights_4)
macro_train_4

0.6714236060956233

In [86]:
macro_base_4 = macro_f1(f1s_base, weight = weights_4)
macro_base_4

0.657539039907461

In [87]:
macro_train8_4 = macro_f1(f1s_train_8, weight = weights_4)
macro_train8_4

0.655673012580638

In [88]:
macro_llama_4 = macro_f1(f1s_llama, weight = weights_4)
macro_llama_4

0.6511992550888828

In [89]:
## Weight to perform prediction 1
first_f4 = ['Weight to perform prediction 1', macro_base_4, macro_train_4, macro_train8_4, macro_llama_4]
df_f1scores.loc[4] = first_f4

In [90]:
variables_train

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'passivating_molecule',
 'efficiency_cont',
 'efficiency_tret',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc']

In [91]:
weights_5 = [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1]

In [92]:
macro_train_5 = macro_f1(f1s_train, weight = weights_5)
macro_train_5

0.6998679896493344

In [93]:
macro_base_5 = macro_f1(f1s_base, weight = weights_5)
macro_base_5

0.6402383306744419

In [94]:
macro_train8_5 = macro_f1(f1s_train_8, weight = weights_5)
macro_train8_5

0.6797206543542185

In [95]:
macro_llama_5 = macro_f1(f1s_llama, weight = weights_5)
macro_llama_5

0.682696690018012

In [96]:
## Weight to perform prediction 2
first_f5 = ['Weight to perform prediction 2', macro_base_5, macro_train_5, macro_train8_5, macro_llama_5]
df_f1scores.loc[5] = first_f5

In [97]:
variables_train

['perovskite_composition',
 'electron_transport_layer',
 'hole_transport_layer',
 'structure_pin_nip',
 'stability_type',
 'temperature',
 'time',
 'humidity',
 'passivating_molecule',
 'efficiency_cont',
 'efficiency_tret',
 'control_pce',
 'treated_pce',
 'control_voc',
 'treated_voc']

In [98]:
weights_6 = [1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0]

In [99]:
macro_train_6 = macro_f1(f1s_train, weight = weights_6)
macro_train_6

0.44105788840461757

In [100]:
macro_base_6 = macro_f1(f1s_base, weight = weights_6)
macro_base_6

0.41121938156864923

In [101]:
macro_train8_6 = macro_f1(f1s_train_8, weight = weights_6)
macro_train8_6

0.5372934239365788

In [102]:
macro_llama_6 = macro_f1(f1s_llama, weight = weights_6)
macro_llama_6

0.507860377430994

In [103]:
## Weight to perform prediction 3
first_f6 = ['Weight to perform prediction 3', macro_base_6, macro_train_6, macro_train8_6, macro_llama_6]
df_f1scores.loc[6] = first_f6

In [104]:
df_f1scores

,Macro F1 score weight distribution,Base Deepseek 8bit,Fine-Tuned Deepseek 4 bit,Fine-Tuned Deepseek 8 bit,Llama 3 billion parameter
0,Macro F1 score with equal weight,0.478965,0.529807,0.590996,0.582662
1,Heavier weight on stability,0.431204,0.478216,0.563622,0.546909
2,Heavier weight on perovskite structure,0.490628,0.540545,0.593914,0.591821
3,Heavier weight on numeric data,0.489202,0.542935,0.592720,0.580230
4,Weight to perform prediction 1,0.657539,0.671424,0.655673,0.651199
5,Weight to perform prediction 2,0.640238,0.699868,0.679721,0.682697
6,Weight to perform prediction 3,0.411219,0.441058,0.537293,0.507860


### Different weight to consider
- Unweight
- Heavier weight on stability
- Heavier weight on perovskite structure
- Heavier weight on numeric data
- Weight to perform prediction 1
- Weight to perform prediction 2
- Weight to perform prediction 3